In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Resizing
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

In [63]:


# Paramètres
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
DATA_DIR = "data"  # structure : data/train / data/val

# 1. Chargement des datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR + "/train",

    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    crop_to_aspect_ratio = True,
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR + "/val",

    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    crop_to_aspect_ratio = True,
)

class_names = train_ds.class_names
num_classes = len(class_names)


Found 4900 files belonging to 5 classes.
Found 100 files belonging to 5 classes.


## Modèle simple

In [64]:


# 2. Pipeline de performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# 3. Couche d’augmentation des données
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])


# Suppose on charge des images de taille variée, e.g. 1000×600
#inputs = layers.Input(shape=(None, None, 3))  # taille variable

#x = Resizing(224, 224,
#             crop_to_aspect_ratio=True,
#             interpolation='bilinear')(inputs)
# x a maintenant la taille (224,224,3) sans déformation ratio


inputs = layers.Input(shape=(*IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# 5. Callbacks
log_dir = "logs"
cb_list = [
    callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    callbacks.TensorBoard(log_dir=log_dir)
]

# 6. Entraînement
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=cb_list
)

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_2 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 200704)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 200704)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │    25,690,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,710,277 (98.08 MB)

 Trainable params: 25,710,277 (98.08 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 177s 883ms/step - accuracy: 0.3833 - loss: 2.1568 - val_accuracy: 0.6600 - val_loss: 0.8225
Epoch 2/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 136s 883ms/step - accuracy: 0.7292 - loss: 0.6957 - val_accuracy: 0.6600 - val_loss: 0.8537
Epoch 3/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 136s 885ms/step - accuracy: 0.8236 - loss: 0.4563 - val_accuracy: 0.7500 - val_loss: 0.4682
Epoch 4/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 130s 843ms/step - accuracy: 0.8623 - loss: 0.3577 - val_accuracy: 0.8600 - val_loss: 0.3405
Epoch 5/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 131s 848ms/step - accuracy: 0.8989 - loss: 0.2812 - val_accuracy: 0.6900 - val_loss: 0.5173
Epoch 6/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 130s 844ms/step - accuracy: 0.9074 - loss: 0.2523 - val_accuracy: 0.7700 - val_loss: 0.4960
Epoch 7/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 133s 865ms/step - accuracy: 0.9233 - loss: 0.1969 - val_accuracy: 0.7800 - val_loss: 0.6083
Epoch 8/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 131s 848ms/step - accuracy: 0.9228 -

## Modèle VGG16 avec modification des couches denses

In [139]:
# Create a pre-trained VGG16 model
pretrained_model = VGG16(
    input_shape=(224, 224, 3),  # Input image shape (224x224 pixels with 3 color channels)
    include_top=False,          # Exclude the fully connected layers (top)
    weights='imagenet',         # Use pre-trained weights from ImageNet
    pooling='avg'               # Use global average pooling as the final layer
)

# Freeze the layers of the pre-trained model
pretrained_model.trainable = False


In [140]:
inputs=pretrained_model.input

x=layers.Dense(128,activation='relu')(pretrained_model.output)

x=layers.Dense(128,activation='relu')(x)

x=layers.Dropout(rate=0.3)(x)

x=layers.BatchNormalization()(x)

x=layers.Dense(64,activation='relu')(x)

outputs=layers.Dense(num_classes,activation='softmax')(x)

# Create the final custom model by specifying the input and output layers
model=Model(inputs=inputs, outputs=outputs)

In [141]:
#pretrained_model = load_model('Fish_Classification_VGG16.h5')

for layer in model.layers:
    print(layer.name, layer.trainable, len(layer.trainable_weights))

print("Total trainable weights :", len(model.trainable_weights))



input_layer_15 False 0
block1_conv1 False 0
block1_conv2 False 0
block1_pool False 0
block2_conv1 False 0
block2_conv2 False 0
block2_pool False 0
block3_conv1 False 0
block3_conv2 False 0
block3_conv3 False 0
block3_pool False 0
block4_conv1 False 0
block4_conv2 False 0
block4_conv3 False 0
block4_pool False 0
block5_conv1 False 0
block5_conv2 False 0
block5_conv3 False 0
block5_pool False 0
global_average_pooling2d_5 False 0
dense_27 True 2
dense_28 True 2
dropout_7 True 0
batch_normalization_4 True 2
dense_29 True 2
dense_30 True 2
Total trainable weights : 10


In [ ]:
#pretrained_model.load_weights('Fish_Classification_VGG16.h5')

In [142]:
model.summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 14,805,957 (56.48 MB)

 Trainable params: 91,013 (355.52 KB)

 Non-trainable params: 14,714,944 (56.13 MB)

In [143]:
    # freeze the base model by making it non trainable
base_learning_rate = 0.001

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [144]:
print(model.summary())

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 14,805,957 (56.48 MB)

 Trainable params: 91,013 (355.52 KB)

 Non-trainable params: 14,714,944 (56.13 MB)

None


In [145]:
for layer in model.layers:
    print(layer.name, layer.trainable, len(layer.trainable_weights))

print("Total trainable weights :", len(model.trainable_weights))

input_layer_15 False 0
block1_conv1 False 0
block1_conv2 False 0
block1_pool False 0
block2_conv1 False 0
block2_conv2 False 0
block2_pool False 0
block3_conv1 False 0
block3_conv2 False 0
block3_conv3 False 0
block3_pool False 0
block4_conv1 False 0
block4_conv2 False 0
block4_conv3 False 0
block4_pool False 0
block5_conv1 False 0
block5_conv2 False 0
block5_conv3 False 0
block5_pool False 0
global_average_pooling2d_5 False 0
dense_27 True 2
dense_28 True 2
dropout_7 True 0
batch_normalization_4 True 2
dense_29 True 2
dense_30 True 2
Total trainable weights : 10


In [ ]:
initial_epochs = 5


# 6. Entraînement
history = model.fit(
    train_ds, 
    validation_data=val_ds,
    epochs=initial_epochs
)

Epoch 1/5
154/154 ━━━━━━━━━━━━━━━━━━━━ 914s 6s/step - accuracy: 0.7139 - loss: 0.7848 - val_accuracy: 1.0000 - val_loss: 0.0534
Epoch 2/5
 17/154 ━━━━━━━━━━━━━━━━━━━━ 13:10 6s/step - accuracy: 0.9747 - loss: 0.0487

AbortedError: Graph execution error:

Detected at node StatefulPartitionedCall/functional_15_1/block3_conv3_1/Relu defined at (most recent call last):
<stack traces unavailable>
Operation received an exception:Status: 1, message: could not create a memory object, in file tensorflow/core/kernels/mkl/mkl_conv_ops.cc:1112
	 [[{{node StatefulPartitionedCall/functional_15_1/block3_conv3_1/Relu}}]] [Op:__inference_multi_step_on_iterator_104027]

In [148]:
tf.__version__

'2.19.0'